## Building the dataset

Based on [this paper][https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf], we build an MLP that trains the embeddings of the characters to learn some semantic meaning behind the letters and how they interact together. 

In [11]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [13]:
len(words)

32033

In [14]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [40]:
block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '->', itos[ix])
        context = context[1:] + [ix] # sliding window: take the last 2 and add the new one to the end

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... -> e
..e -> m
.em -> m
emm -> a
mma -> .
olivia
... -> o
..o -> l
.ol -> i
oli -> v
liv -> i
ivi -> a
via -> .
ava
... -> a
..a -> v
.av -> a
ava -> .
isabella
... -> i
..i -> s
.is -> a
isa -> b
sab -> e
abe -> l
bel -> l
ell -> a
lla -> .
sophia
... -> s
..s -> o
.so -> p
sop -> h
oph -> i
phi -> a
hia -> .


In [41]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

## Embeddings layer

![image](images/MLP_arch.png)

In the paper, they reduce 17,000 words into 30D vectors. We can reduce our 27 characters to a 2D space. `C` is the lookup table.

In [42]:
C = torch.randn((27, 2), requires_grad=True)
C.shape

torch.Size([27, 2])

In [43]:
C[5]

tensor([-0.5778, -0.4449], grad_fn=<SelectBackward0>)

In [44]:
C[[0, 1, 6, 26]]

tensor([[-0.7610,  1.0278],
        [ 0.6214, -1.2836],
        [-1.0222,  0.8242],
        [ 0.7839, -0.2145]], grad_fn=<IndexBackward0>)

In [50]:
X

# ['.', '.', '.']
# ['.', '.', 'e']
# .
# .
# .

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [49]:
X.shape

torch.Size([32, 3])

In [51]:
C[X].shape

# ['.', '.', '.'] -> [2D embedding, 2D embedding, 2D embedding]
# For each of the 32 instances in X

torch.Size([32, 3, 2])

In [53]:
X[3, 2]

# 3rd instance: ['e', 2nd char: 'm', 'm'] -> 'm' -> 13

tensor(13)

In [55]:
print(C[X][3, 2])
print(C[13])

tensor([0.0803, 0.9627], grad_fn=<SelectBackward0>)
tensor([0.0803, 0.9627], grad_fn=<SelectBackward0>)


In [59]:
emb_X = C[X]
emb_X.shape

torch.Size([32, 3, 2])

## Hidden layer (first linear layer in MLP)

Our model has a block size of 3, so 3 2D vectors are the input to this layer -> 6D i/p

In [58]:
W1 = torch.randn((6, 100), requires_grad=True) # 100 neurons in the hidden layer
b1 = torch.randn((100), requires_grad=True)

In [61]:
emb_X.shape, W1.shape, b1.shape

(torch.Size([32, 3, 2]), torch.Size([6, 100]), torch.Size([100]))

In [60]:
emb_X @ W1 + b1

RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [68]:
# Embeddings of the first char (3 block size) of each instance
emb_X[:, 0, :].shape

torch.Size([32, 2])

In [72]:
torch.cat([emb_X[:, 0, :], emb_X[:, 1, :], emb_X[:, 2, :]], dim=1).shape

# dim = 0 will add to rows and then the shape will be (94, 2)


torch.Size([32, 6])

We can use unbind to this dynamically and work for any block size

In [82]:
ex = torch.randn((32, 3, 2))
len(torch.unbind(ex, dim=0))

# 32 of (3, 2): [[[1, 1], [1, 2], [27, 32]], \
# [[1, 1], [1, 2], [27, 32]], \
# [[1, 1], [1, 2], [27, 32]]] -> 32 of (1): [[1, 1], [1, 2], [27, 32], [1, 1], [1, 2], [27, 32], [1, 1], [1, 2], [27, 32]]

# Unpacked about the 0th dimension

32

In [95]:
len(torch.unbind(ex, dim=1))

3

In [99]:
ex[0]

tensor([[ 0.8502,  0.3931],
        [-1.4049,  0.8801],
        [ 0.5862,  0.5581]])

In [101]:
ex[1]

tensor([[ 0.8983, -0.9280],
        [ 0.5852,  1.3669],
        [-0.1991,  0.7075]])

In [100]:
torch.unbind(ex, dim=1)[0] # First chars (total 3) of each instance -> 32, each 2D

tensor([[ 0.8502,  0.3931],
        [ 0.8983, -0.9280],
        [-1.0248,  1.1669],
        [ 0.2538,  0.5970],
        [ 1.6937, -0.0551],
        [-1.0495,  0.2372],
        [-0.7514, -1.0041],
        [ 1.1320,  0.7583],
        [-0.6416, -0.6086],
        [-0.9542, -1.7272],
        [ 0.7051,  1.8348],
        [-0.7262, -0.3246],
        [-1.3134,  0.1161],
        [ 0.6041,  0.3985],
        [-0.2767, -0.7103],
        [-0.6970, -0.0549],
        [-0.4348,  0.1049],
        [-0.2195,  0.5332],
        [ 2.7029,  0.2255],
        [-0.7859, -0.2080],
        [-0.6512, -0.6692],
        [ 0.7188, -0.5136],
        [-0.0715, -1.1498],
        [ 1.9478, -0.6732],
        [ 0.6553,  0.1066],
        [ 0.9181, -0.8961],
        [ 0.2918, -0.6176],
        [-0.3471,  1.1438],
        [ 0.5121,  0.1543],
        [ 1.9462,  0.0998],
        [-0.5453,  0.6782],
        [-0.0595, -0.5498]])

In [98]:
torch.unbind(ex, dim=1)[0].shape

torch.Size([32, 2])

In [94]:
torch.cat(torch.unbind(ex, dim=1), dim=1).shape

# same as torch.cat([emb_X[:, 0, :], emb_X[:, 1, :], emb_X[:, 2, :]], dim=1).shape but for any block size

torch.Size([32, 6])

We can use `torch.view` for more simplicity and efficiency

In [102]:
ex2 = torch.randn(18)
ex2.storage()

 2.175413131713867
 -0.0892491340637207
 -1.3933746814727783
 -0.19329167902469635
 0.6051356196403503
 0.6366099715232849
 -1.4114915132522583
 -0.7883928418159485
 -0.22982749342918396
 -1.0874011516571045
 0.32152432203292847
 2.5040786266326904
 -1.125294804573059
 0.11302115768194199
 0.9527454972267151
 -1.152559518814087
 0.361823171377182
 0.6077340245246887
[torch.FloatStorage of size 18]

Tensors are always stored as a group of numbers in memory. We can use `torch.view()` to pack them as we desire.

In [104]:
ex2.view(3, 6)

tensor([[ 2.1754, -0.0892, -1.3934, -0.1933,  0.6051,  0.6366],
        [-1.4115, -0.7884, -0.2298, -1.0874,  0.3215,  2.5041],
        [-1.1253,  0.1130,  0.9527, -1.1526,  0.3618,  0.6077]])

In [109]:
emb_X.view(32, 6) == torch.cat(torch.unbind(emb_X, 1), 1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

Finally,

In [112]:
h = torch.tanh(emb_X.view(emb_X.shape[0], 6) @ W1 + b1)
h.shape # activations of 100 neurons for each of the 32 instances

torch.Size([32, 100])

Even more dynamic,

In [113]:
h = torch.tanh(emb_X.view(-1, 6) @ W1 + b1) # PyTorch will infer the required size based on that dim 1 = 6
h.shape

torch.Size([32, 100])

## Output

We can build layer 2 now. This is the output layer so we need 27 neurons here (each taking the 100D o/p of previous layer as i/p) to get a prob dist for the next char given a context window of block size 3.

In [114]:
W2 = torch.randn((100, 27), requires_grad=True)
b2 = torch.randn((27), requires_grad=True)